### Prepare the data to train the Skip-Aware Fast-Forward Agent (SAFFA)

- Clone the repo and install the dependencies (you may need to restart the session after running the following command once):

In [ ]:
! git clone https://github.com/verlab/TextDrivenVideoAcceleration_TPAMI_2022
%cd TextDrivenVideoAcceleration_TPAMI_2022
! pip install -r requirements.txt

- To train the agent, you will need the features produced the VDAN+ model. You can have these features using:

In [ ]:
%cd rl_fast_forward

# Download YouCook2's VDAN+ video feats
! wget -O resources/YouCook2/VDAN+/youcook2_vdan+_vid_feats.zip https://verlab.dcc.ufmg.br/TextDrivenVideoAcceleration/youcook2_vdan+_vid_feats.zip
! unzip -q resources/YouCook2/VDAN+/youcook2_vdan+_vid_feats.zip -d resources/YouCook2/VDAN+/vid_feats/
! rm resources/YouCook2/VDAN+/youcook2_vdan+_vid_feats.zip

# Download YouCook2's VDAN+ document feats
! wget -O resources/YouCook2/VDAN+/youcook2_vdan+_doc_feats.zip https://verlab.dcc.ufmg.br/TextDrivenVideoAcceleration/youcook2_vdan+_doc_feats.zip
! unzip -q resources/YouCook2/VDAN+/youcook2_vdan+_doc_feats.zip -d resources/YouCook2/VDAN+/doc_feats/
! rm resources/YouCook2/VDAN+/youcook2_vdan+_doc_feats.zip

- If you want to extract them by yourself, you can have a VDAN pretrained model as follows:

In [ ]:
# Download the pretrained model
%cd /content/TextDrivenVideoAcceleration_TPAMI_2022
! wget -O semantic_encoding/models/vdan+_pretrained_model.pth https://verlab.dcc.ufmg.br/TextDrivenVideoAcceleration/vdan+_model_pretrained.pth

- Now, prepare the data for training...
  1. First, download the annotations;
  2. Then, download the videos, IF NECESSARY. **PS.: If you already have the VDAN features extracted, skip this step**;
  3. Finally, prepare the recipe files.

In [ ]:
# 1. Download and extract the annotations
%cd /content/TextDrivenVideoAcceleration_TPAMI_2022
! wget -O rl_fast_forward/resources/YouCook2/youcookii_annotations_trainval.tar.gz http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
! tar -xf rl_fast_forward/resources/YouCook2/youcookii_annotations_trainval.tar.gz -C rl_fast_forward/resources/YouCook2/
! rm rl_fast_forward/resources/YouCook2/youcookii_annotations_trainval.tar.gz

In [ ]:
# 2. Download the scripts used to collect the videos + Install youtube-dl and download the available videos
! wget -O rl_fast_forward/resources/YouCook2/scripts.tar.gz http://youcook2.eecs.umich.edu/static/YouCookII/scripts.tar.gz
! tar -xf rl_fast_forward/resources/YouCook2/scripts.tar.gz -C rl_fast_forward/resources/YouCook2/
! rm rl_fast_forward/resources/YouCook2/scripts.tar.gz

! pip install youtube_dl
%cd rl_fast_forward/resources/YouCook2/scripts
! python download_youcookii_videos.py

In [ ]:
# 3. Prepare the recipe files and feature folders
%cd /content/TextDrivenVideoAcceleration_TPAMI_2022/rl_fast_forward/
! python resources/create_youcook2_recipe_documents.py

- You are set! Now, you just need to run it...

In [ ]:
! python train.py -s ../semantic_encoding/models/vdan+_pretrained_model.pth -d YouCook2

- After training, the model will be saved in the [rl_fast_forward/models](rl_fast_forward/models) folder. Then, the script will generate a results JSON file with the pattern `results/<datetime>_<hostname>_youcookii_selected_frames.json`

- To evaluate the results, download the [YouCook2's ground-truth JSON file](https://github.com/verlab/TextDrivenVideoAcceleration_TPAMI_2022/releases/download/v1.0.0/youcookii_gts.json), then run the evaluation script.

In [ ]:
! wget https://verlab.dcc.ufmg.br/TextDrivenVideoAcceleration/youcookii_gts.json
! python eval/eval_results.py -gt eval/youcookii_gts.json -sf results/<datetime>_<hostname>_youcookii_selected_frames.json

- If you want to train using the COIN dataset, follow the same steps changing the folder names and annotation files accordingly. Please note that, to train in the COIN dataset you must also provide the domain on which you want to train the agent using the option -dm <DOMAIN_ACRONYM>. E.g., `python train.py -s ../semantic_encoding/models/vdan+_pretrained_model.pth -d DOIN -dm NC` for the domain Nursing and Care.